# Regression ANN with best parameters
    find the best prediction window to apply to 2 years data with a rolling Nested CV technique

In [15]:
import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

dates = [2017000000, 2017030000, 2017060000, 2017090000, 2018000000, 2018030000, 2018060000, 2018090000]
dates_labels = ['2 years', '1 year and a half', '1 year', '6 months']

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

SyntaxError: invalid syntax (<ipython-input-15-b4de82005957>, line 9)

# Loop with model for 4 different data ranges (6 months intervals):

In [ ]:
for i in dates:
    count = 0
    # data
    data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)
    data = data.loc[data.index > i, :]
    
    # reset index
    data.reset_index(inplace = True)
    data.drop('index', axis = 1, inplace = True)
    
    # Divide features and labels
    X = data.iloc[:, 0:15]
    y = data.loc[:, 'Offers']
    
    X.fillna(X.mean(), inplace = True)
    y.fillna(y.mean(), inplace = True)
    
    X = X.astype('float64')
    X = X.round(20)
    
    # divide data into train and test with 20% test data
    X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size=0.2, shuffle=False)
    
    # feature scaling
    sc_X = MinMaxScaler()
    X_train = sc_X.fit_transform(X_train)
    X_test = sc_X.transform(X_test)
    
    import keras
    from keras.models import Sequential # to initialise the NN
    from keras.layers import Dense # to create layers
    from keras.layers import Dropout
    from keras import initializers
    import keras.optimizers
    from keras.wrappers.scikit_learn import KerasRegressor
    
    # possible debug
    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)
    
    def regressor_tunning(n_hidden = 2, 
                          n_neurons = 30, 
                          optimizer = 'Adamax', 
                          kernel_initializer = "he_normal",
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        model.add(Dense(units = n_neurons, input_dim = 15))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.1))
        for layer in range(n_hidden):
            model.add(Dense(n_neurons))
            model.add(keras.layers.LeakyReLU(alpha = 0.2))
            model.add(Dropout(rate = 0.1))
        model.add(Dense(units = 1, activation = 'linear'))
        model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
        return model
    
    tscv = TimeSeriesSplit(n_splits = 7)
    
    hist_list = pd.DataFrame()
    count = 1
    
    regressor = regressor_tunning()
    
    for train_index, test_index in tscv.split(X_train):
          X_train_split, X_test_split = X_train[train_index], X_train[test_index]
          y_train_split, y_test_split = y_train[train_index], y_train[test_index]
          hist = regressor.fit(X_train_split, y_train_split, batch_size = 15, epochs = 80)
          hist_list = hist_list.append(hist.history, ignore_index = True)
          print(count)
          count = count + 1

    a = []
    b = []
    
    for i in range(len(hist_list.mse)):
        a.append(np.mean(hist_list.mse[i]))
        b.append(np.mean(hist_list.mae[i]))
      
    mse_cv.append(np.mean(a))
    mae_cv.append(np.mean(b))
    
    # predict for X_test  
    y_pred = regressor.predict(X_test)

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mse_error = mse(y_test, y_pred) # 1479.61335
    mae_error = mae(y_test, y_pred) # 23.1525
    
    rmse_gen.append(rmse_error)
    mse_gen.append(mse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values
    
    
    # smal adjustment
    y_test.replace(0, 0.0001,inplace = True)

    
    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mse_spike = mse(y_test_spike, y_pred_spike)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mse_spi.append(mse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mse_normal = mse(y_test_normal, y_pred_normal)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mse_nor.append(mse_normal)
    mae_nor.append(mae_normal)

/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Epoch 1/80
3507/3507 [==============================] - 1s 399us/step - loss: 9354.2266 - mse: 9354.2246 - mae: 78.5116
Epoch 2/80
3507/3507 [==============================] - 1s 159us/step - loss: 2365.8247 - mse: 2365.8247 - mae: 33.1048
Epoch 3/80
3507/3507 [==============================] - 1s 158us/step - loss: 2325.2546 - mse: 2325.2544 - mae: 33.0188
Epoch 4/80
3507/3507 [==============================] - 1s 148us/step - loss: 2258.5323 - mse: 2258.5320 - mae: 32.8474
Epoch 5/80
3507/3507 [==============================] - 1s 156us/step - loss: 2235.4748 - mse: 2235.4753 - mae: 32.7039
Epoch 6/80
3507/3507 [==============================] - 1s 155us/step - loss: 2211.1425 - mse: 2211.1428 - mae: 32.5188
Epoch 7/80
3507/3507 [==============================] - 1s 158us/step - loss: 2233.0266 - mse: 2233.0271 - mae: 32.7050
Epoch 8/80
3507/3507 [==============================] - 1s 153us/step - loss: 2233.4681 - mse: 2233.4683 - mae: 32.3103
Epoch 9/80
3507/3507 [==================